# Import Scripts and Libraries

In [ ]:
import norm_heatmap
import normalize
import tools
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import heatmap_workflow_tools as hwt
import os
import cv2

# 1 - Setting up Filesystem
### *User input required!*

Before running the code, please ensure your filesystem consists of a parent directory with the ProDiVis repository and any dataset repositories cloned within. An example filesystem could look like the following:


`📁<parent_directory>`
```
└── 📁ProDiVis
    └──📁Scripts 
        └── Data_Pipeline.ipynb
        └── *.py
    └──📁data
        └──📁signal_stack
        └──📁normalization_stack
```

`└── 📁ProDiVis_Glioblastoma` <span style="color:green"><em># clone if desired</em></span>

`└── 📁Mouse_Hearts` <span style="color:green"><em># clone if desired</em></span>

The repositories containing the glioblastoma and mouse heart datasets can be found at the links below:
 - [**ProDiVis_Glioblastoma**]()
 - [**Mouse_Hearts**]()


In [ ]:
# In quotes, type the path to the ProDiVis repository:
WORKING_DIRECTORY = "/home/asathler/franco/ProDiVis"

# 2. Pre-Analysis

In [ ]:

stack_dir = os.path.join(WORKING_DIRECTORY, "sample_data/green_soi/")
norm_dir = os.path.join(WORKING_DIRECTORY, "sample_data/blue_ns/")

# What is your signal of interest (SOI) and normalization signal (NS) 
soi = "Name of SOI"
ns = "Name of NS"

# Replace with slice integers or decimal percentages. 'None' for no boundary
zmin, zmax = None, None

# Do you want to multiply the pixels in the front and side views (int)?
z_multiplier =  10

# Do you want to save your figures? (boolean)
save_figs = False

# Adjust the lower and upper threshold (int)
lower_thresh = 0
upper_thresh = 254

In [ ]:
stack_tiffs = tools.get_files(stack_dir)
norm_tiffs = tools.get_files(norm_dir)
zmin, zmax = hwt.process_zmin_zmax(zmin, zmax, stack_tiffs)
print(f"Processed ZMin and ZMax: {zmin}, {zmax}")
stack_name = os.path.basename(stack_dir)
norm_name = os.path.basename(norm_dir)
base_dir = os.path.basename(os.path.dirname(norm_dir))
out_dir = os.path.abspath(os.path.join(os.path.abspath(''), '..', 'exported_data'))
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
out_dir = os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_out")
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

# Begin Analysis

### Determine Images Above Threshold

#### Normalization Signal

In [ ]:
norm_imgs = normalize.analyze_images(norm_dir, lower_thresh)
norm_imgs = normalize.images_used(norm_imgs, norm_tiffs, stack_tiffs)

#### Signal of Interest

In [ ]:
soi_imgs = normalize.analyze_images(stack_dir, lower_thresh)
soi_imgs = normalize.images_used(soi_imgs, norm_tiffs, stack_tiffs)

# Visualise Image Histograms and Distribution

#### SOI Depth Loss

In [ ]:
nbins = 10
histograms = np.zeros((len(stack_tiffs[zmin:zmax]), nbins))
for idx, img in enumerate(stack_tiffs[zmin:zmax]):
    tiff = normalize.thresh(img, lower_thresh, upper_thresh, 1)
    hist, bins = np.histogram(tiff, bins = nbins)
    histograms[idx] = hist
histograms[histograms == 0] = 1
bin_labels = ["{}-{}".format(round(bins[i-1],1), round(bins[i],1)) for i in range(1, len(bins))]
hwt.plot_histograms(histograms, f"{base_dir} {stack_name} z{zmin}-{zmax} SOI Heatmap of Histograms",
os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_SOI_Histograms.pdf"), bin_labels, save_figs)

In [ ]:
stack_means = np.array([])
for img in stack_tiffs[zmin:zmax]:
    mean = normalize.tiff_stats_thresh(img, lower_thresh, upper_thresh, 1)
    stack_means = np.append(stack_means, mean)
hwt.plot_MSI(stack_means,
             f"{base_dir} {stack_name} z{zmin}-{zmax}",
             os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_SOI_Intensity.pdf"), save_figs)

### NS Depth Loss

In [ ]:
nbins = 10
histograms = np.zeros((len(stack_tiffs[zmin:zmax]), nbins))
for idx, img in enumerate(norm_tiffs[zmin:zmax]):
    tiff = normalize.thresh(img, 0, 254, 1)
    hist, bins = np.histogram(tiff, bins = nbins)
    histograms[idx] = hist
histograms[histograms == 0] = 1
bin_labels = ["{}-{}".format(round(bins[i-1],1), round(bins[i],1)) for i in range(1, len(bins))]
hwt.plot_histograms(histograms, f"{base_dir} {stack_name} z{zmin}-{zmax} NS Heatmap of Histograms",
os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_NS_Histograms.pdf"), bin_labels, save_figs)

In [ ]:
norm_means = np.array([])
for norm_tiff in norm_tiffs[zmin:zmax]:
    mean = normalize.tiff_stats_thresh(norm_tiff, 0, 254, 1)
    norm_means = np.append(norm_means, mean)
hwt.plot_MSI(norm_means,
             f"{base_dir} {norm_name} z{zmin}-{zmax}",
             os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_NS_Intensity.pdf"), save_figs)

### NS vs SOI

In [ ]:
hwt.plot_MSI_soi_ns(soi, ns, stack_means, norm_means, f"{base_dir} {stack_name} {norm_name} z{zmin}-{zmax}",
             os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_z{zmin}-{zmax}_SOI_vs_NS.pdf"), save_figs)

# SNIP Raw Heatmap

In [ ]:
imgs = norm_heatmap.stack(stack_tiffs[zmin:zmax], ['z', 'x', 'y'], z_multiplier)
hwt.plot_MIP(imgs, f"{base_dir} {stack_name} z{zmin}-{zmax}", 
                os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_HeatMap.pdf"), save_figs)

# Normalization

In [ ]:
norm_stack_tiffs = normalize.mean_normalizer(stack_tiffs[zmin:zmax], norm_tiffs[zmin:zmax], lower_thresh, upper_thresh, True, False)

#### Pixel Distribution Post-Normalization

In [ ]:
norm_stack_means = np.array([])
for norm_stack_tiff in norm_stack_tiffs[zmin:zmax]:
    mean = normalize.tiff_stats_thresh(norm_stack_tiff, 0, 255, 1)
    norm_stack_means = np.append(norm_stack_means, mean)
hwt.plot_MSI(norm_stack_means,
             f"{base_dir} {stack_name} norm {norm_name} z{zmin}-{zmax}",
             os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_z{zmin}-{zmax}_Normalized_SOI_Intensity.pdf"), save_figs)

hwt.plot_MSI_grouped(stack_means, norm_means, norm_stack_means, f"{base_dir} {stack_name} norm {norm_name} z{zmin}-{zmax}_Grouped",
                os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_z{zmin}-{zmax}_Normalized_Intensity_grouped.pdf"), save_figs)

# SNIP Normalized Heatmap

In [ ]:
imgs = norm_heatmap.stack(norm_stack_tiffs[zmin:zmax], ['z', 'x', 'y'], z_multiplier)
hwt.plot_MIP(imgs, f"{base_dir} {stack_name} norm {norm_name} z{zmin}-{zmax}", 
                os.path.join(out_dir, f"{base_dir}_{stack_name}n{stack_name}_z{zmin}-{zmax}_Normalized_HeatMap.pdf"), save_figs)

# Image Rescaling

In [ ]:
tools.stack_gui(norm_stack_tiffs, norm_tiffs)

#### Insert Path to Scaled Images

In [ ]:
scaled_norm_stack_tiffs = os.path.abspath("/Users/Kyle/Desktop/prodivis_public_repo/prodivis/sample_data/scaled_green_soi_rn_blue_ns_254std_nblue_ns")
scaled_norm_stack_tiffs = tools.get_files(scaled_norm_stack_tiffs)

#### Pixel Distribution of Scaled Images

In [ ]:
nbins = 10
histograms = np.zeros((len(scaled_norm_stack_tiffs[zmin:zmax]), nbins))
for idx, img in enumerate(scaled_norm_stack_tiffs[zmin:zmax]):
    tiff = normalize.thresh(img, 0, 255, 1)
    hist, bins = np.histogram(tiff, bins = nbins)
    histograms[idx] = hist
histograms[histograms == 0] = 1
bin_labels = ["{}-{}".format(round(bins[i-1],1), round(bins[i],1)) for i in range(1, len(bins))]
hwt.plot_histograms(histograms, f"{base_dir} {stack_name} z{zmin}-{zmax} Normalized SOI Heatmap of Histograms",
os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_Scaled_Normalized_SOI_Histograms.pdf"), bin_labels, save_figs)

#### Normalized Depth Loss of Scaled Images

In [ ]:
scaled_norm_stack_means = np.array([])
for scaled_norm_stack_tiff in scaled_norm_stack_tiffs[zmin:zmax]:
    mean = normalize.tiff_stats_thresh(scaled_norm_stack_tiff, 0, 255, 1)
    scaled_norm_stack_means = np.append(scaled_norm_stack_means, mean)
hwt.plot_MSI(scaled_norm_stack_means,
             f"Scaled_{base_dir} {stack_name} norm {norm_name} z{zmin}-{zmax}_",
             os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_z{zmin}-{zmax}_Scaled_Normalized_SOI_Intensity.pdf"), save_figs)

# SNIP Normalized Heatmap of Scaled Images

In [ ]:
scaled_imgs = norm_heatmap.stack(scaled_norm_stack_tiffs[zmin:zmax], ['z', 'x', 'y'], z_multiplier)
hwt.plot_MIP(scaled_imgs, f"{base_dir} {stack_name} norm {norm_name} z{zmin}-{zmax}", 
                os.path.join(out_dir, f"{base_dir}_{stack_name}n{stack_name}_z{zmin}-{zmax}_Scaled_Normalized_HeatMap.pdf"), save_figs)